In [16]:
import pandas as pd
import skbio
from functools import partial
from multiprocessing import Pool
from skbio import TreeNode
from skbio.diversity.alpha import faith_pd
from skbio.diversity.beta import unweighted_unifrac, weighted_unifrac

In [2]:
# global variables go in all caps
MYBIOM = pd.read_table('./updated_lacto_sum_scaled.tsv')
MYTREE = skbio.read('./tree.nwk', format='newick', into=TreeNode)

NameError: name 'mybiom' is not defined

In [6]:
SAMPLE_IDS = MYBIOM.columns[1:]
OTU_IDS = MYBIOM['asv']
NCORES = 6

In [7]:
def faith_fun(sample_id):
    # to avoid doing "partial" nonsense, we wrap the faith_pd
    # function in another function so we can reference the 
    # global OTU_IDS and MYTREE variables.

    # get the column for the given sample_id
    sample = MYBIOM[sample_id]
    # run faith_pd on sample
    alpha = faith_pd(sample, otu_ids=OTU_IDS, tree=MYTREE)

    return alpha

In [10]:
with Pool(NCORES) as p:
    alphas = p.map(faith_fun, SAMPLE_IDS)

# convert to dataframe for export
alphas = pd.DataFrame({'sampleid' : SAMPLE_IDS,
                       'faith_pd' : alphas})

In [14]:
combs = []

,sampleid,faith_pd
0,1_0403_9699,41.863220
1,1_0410_9686,47.256110
2,1_0410_9687,39.344774
3,1_0410_9689,44.571748
4,1_0410_9691,40.486682
...,...,...
326,EW.WD,12.444276
327,EW.WD.Fiber,10.852056
328,Mock_6,10.968919
329,Mock20220615A,26.573202


In [25]:
def unweighted_unifrac_fun(sample_id):
    # return all unweighted unifrac distances
    # for a given sample_id (u)

    u = MYBIOM[sample_id]
    beta_list = []
    # iterate over all samples (v) and compare them to u
    for v_id in SAMPLE_IDS:
        # skip expensive calculations if self to self comparison
        if sample_id == v_id:
            beta_list.append(0)
        else:
            v = MYBIOM[v_id]
            beta = unweighted_unifrac(u, v, tree=MYTREE,
                                      otu_ids=OTU_IDS, validate=False)
            beta_list.append(beta)
        
    return beta_list


with Pool(NCORES) as p:
    uw_betas = p.map(unweighted_unifrac_fun, SAMPLE_IDS)

uw = pd.DataFrame(uw_betas)


KeyboardInterrupt: 